In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
# load a file that uses , as delimiter
def load_file(path, names):
    if not path.is_file():
        raise FileNotFoundError(str(path))
    data = pd.read_csv(path, sep=",", names=names, header=None)
    return data

In [3]:
def load_df():
    cols = ["state", "emission"]
    file = Path.cwd() / "data.txt"
    return load_file(file, cols)

In [4]:
data = load_df()
data

,state,emission
0,foggy,no
1,foggy,no
2,foggy,no
3,rainy,yes
4,sunny,no
5,foggy,no
6,rainy,yes
7,rainy,yes
8,foggy,no
9,rainy,yes


In [29]:
def emissionProbabilityMatrix(data):
    matrix = data
    #make column for sunny yes, foggy yes, rainy yes
    #for sunny, foggy, and rainy put 1 in the row if true
    matrix['sunny-yes'] = np.where((matrix['state'] == 'sunny') & (matrix['emission'] == 'yes'),1,0) 
    matrix['foggy-yes'] = np.where((matrix['state'] == 'foggy') & (matrix['emission'] == 'yes'),1,0)
    matrix['rainy-yes'] = np.where((matrix['state'] == 'rainy') & (matrix['emission'] == 'yes'),1,0)
    matrix['sunny'] = np.where(matrix['state'] == 'sunny',1,0) 
    matrix['foggy'] = np.where(matrix['state'] == 'foggy',1,0) 
    matrix['rainy'] = np.where(matrix['state'] == 'rainy',1,0) 
    #drop state and emission
    matrix = matrix.drop(['state','emission'],axis=1) 
    #adds all the ones in the columns
    matrix = matrix.sum(axis=0)
    #calculate probablities of each state
    SY=matrix[0]/matrix[4]
    SN= 1-SY
    FY=matrix[1]/matrix[3]
    FN=1-FY
    RY=matrix[2]/matrix[5]
    RN=1-RY
    #create a new dataframe of probablities 
    matrixData =[[SY,SN],[FY,FN],[RY,RN]]
    matrix = pd.DataFrame(matrixData,columns=['yes','no'], index=['sunny','foggy','rainy'])
    return matrix
emProbMatrix = emissionProbabilityMatrix(data)
emProbMatrix

,yes,no
sunny,0.085020,0.914980
foggy,0.302682,0.697318
rainy,0.812245,0.187755
